In [104]:
import pandas as pd
import numpy as np
import spacy
nlp = spacy.load("en_core_web_sm")

# remove misformatted row due to bad html
raw_df = (
    pd.read_csv('data/raw/quakenu_raw.csv')
        .drop(16803, axis=0)
)

# Use 3 decimal places in output display
pd.set_option("display.precision", 3)

# Set max rows displayed in output to 25
pd.set_option("display.max_rows", 25)
# print(raw_df[:1])
# print(raw_df.dtypes)

In [118]:
import datetime

cleaning_test_df = raw_df[::1000] # every 1000th post

def clean_post_df(raw_df):
    clean_df = raw_df
    clean_df['date'] = pd.to_datetime(clean_df['date'], format='%Y-%m-%d, %H:%M')
    return clean_df

clean_test_df = clean_post_df(cleaning_test_df)

# remove misformatted row due to bad html
raw_df = raw_df.drop(16803, axis=0)
print(raw_df.iloc[16801])

author                                                                                                  94
date                                                                                     2011-07-01, 19:14
forum                                                                                                   22
post_id                                                                                              62862
post_text    to shub-niggurath, i salute you! (and to the rest of you; don't forget to register for qhlan)
thread                                                                                                5272
Name: 16801, dtype: object


C:\Users\BYRONA~1\AppData\Local\Temp/ipykernel_7628/1494892598.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_df['date'] = pd.to_datetime(clean_df['date'], format='%Y-%m-%d, %H:%M')


In [37]:
import re
raw_text_col = raw_df['post_text'].str.replace('[^\w\']', ' ', flags=re.UNICODE)

C:\Users\BYRONA~1\AppData\Local\Temp/ipykernel_7628/2140873254.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  raw_text_col = raw_df['post_text'].str.replace('[^\w\']', ' ', flags=re.UNICODE)


In [43]:
# print(raw_df.columns)
print(raw_df.groupby('author')['post_text'].count())

author
0          4
1        733
100       22
1000       8
10000      2
        ... 
9984       2
999        2
9993       3
9995       4
9996      10
Name: post_text, Length: 2106, dtype: int64


In [5]:
"""
Text gathering/cleaning issues
    - scraper not catching underlines, italics, etc https://www.quakeworld.nu/forum/topic/28
    - bb-code divs https://www.quakeworld.nu/forum/topic/28
    - 
"""

pd.set_option('display.max_colwidth', None)
"""
Cleaning a single string for NLP
    
    - remove nonchars except for apostrophes

"""

print(raw_df[['thread', 'post_id', 'post_text']].sample(n=5))
# print(raw_text_col.sample(n=5))
# print([text for text in raw_text_col.sample(n=15)])

       thread post_id  \
4776      119     762   
67070    2741   35637   
63183      53   36192   
76424    6152   95352   
14464    1344   22277   

                                                                                                                                                          post_text  
4776                                                                       hahahhahahhahaha  :lol:     :lol:     :lol:     :lol:     :lol:     :lol:     :lol:       
67070  then the question comes up which bugs we should actually fix and which ones to not fix. when you think about it, most of the good stuff in quake are...bugs.  
63183                                                                                                                                 omg need that texture set ^_^  
76424                      mick, it's not the fact that the stairs are fake that bothers me, more the ramps for speed, as i thought it would make the map too fast   
14464              

In [28]:
import unidecode

test_accent_str = "Sí señor, ¿qué necesita?"

print(unidecode.unidecode(test_accent_str))

test_lemma_str = "I have question. joe many liberals does it take to change a log by bolb? none. their too bussy. they're gender?"

test_lemma_doc = nlp(test_lemma_str)

# for token in test_lemma_doc:
#     print(token.is_sent_start) 

print(' '.join([token.lemma_ for token in test_lemma_doc]))
# print(test_lemma_doc.cats)

Si senor, ?que necesita?
I have question . joe many liberal do it take to change a log by bolb ? none . their too bussy . they be gender ?
